In [ ]:
#my imports
import numpy as np
import unicodedata
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import string
from nltk import pos_tag
import pandas as pd
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option('display.max_colwidth', -1)
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  del sys.path[0]


In [ ]:
low_ratings = pd.read_csv("https://raw.githubusercontent.com/success81/Yelp_Handyman_Reviews/main/final_low.csv")
high_ratings = pd.read_csv("https://raw.githubusercontent.com/success81/Yelp_Handyman_Reviews/main/final_high.csv")

In [ ]:
two_list = []
my_counter = 0
while my_counter != 3728:
  two_list.append(2)
  my_counter += 1

high_list = []
my_counter = 0
while my_counter != 3728:
  high_list.append("positive")
  my_counter += 1



In [ ]:
one_list = []
my_counter = 0
while my_counter != 1198:
  one_list.append(1)
  my_counter += 1

low_list = []
my_counter = 0
while my_counter != 1198:
  low_list.append("negative")
  my_counter += 1



In [ ]:
#adding ratings to data frame
low_ratings.insert(2, "Rating", one_list, True)
high_ratings.insert(2, "Rating", two_list, True)

In [ ]:
#Changing names of columns and dropping additional column
low_ratings.columns = ["DropMe", "Reviews", "Ratings"]
high_ratings.columns = ["DropMe", "Reviews", "Ratings"]


In [ ]:
#Dropping extra Column
low_ratings.drop("DropMe", axis = 1, inplace = True)
high_ratings.drop("DropMe", axis = 1, inplace = True)

In [ ]:
#Adding Ranking
low_ratings.insert(2, "Ranking", low_list, True)
high_ratings.insert(2, "Ranking", high_list, True)

In [ ]:
high_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3728 entries, 0 to 3727
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Reviews  3728 non-null   object
 1   Ratings  3728 non-null   int64 
 2   Ranking  3728 non-null   object
dtypes: int64(1), object(2)
memory usage: 87.5+ KB


In [ ]:
big_count = 10
big_delete = []
while big_count != 2519:
  big_delete.append(big_count)
  big_count += 1 

In [ ]:
my_test = [6,7,8]
high_ratings.drop(big_delete, axis=0, inplace=True, )

In [ ]:
high_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1219 entries, 0 to 3727
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Reviews  1219 non-null   object
 1   Ratings  1219 non-null   int64 
 2   Ranking  1219 non-null   object
dtypes: int64(1), object(2)
memory usage: 38.1+ KB


In [ ]:
#Combining the two dataframes
total_ratings = pd.concat([low_ratings, high_ratings], axis = 0)

In [ ]:
import nltk
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [ ]:
#taking low ratings dataframe and turning it into list
low_ratings_list = []
for index,val in low_ratings.iterrows():
  low_ratings_list.append(val[0])

In [ ]:
#taking high ratings dataframe and turning it into list
high_ratings_list = []
for index,val in high_ratings.iterrows():
  high_ratings_list.append(val[0])

In [ ]:
#Creating Sentence tokens high ratings
high_sentence_tokens = []
for index,val in high_ratings.iterrows():
  high_sentence_tokens.append(sent_tokenize(val[0]))

In [ ]:
#Creating Sentence tokens low ratings
low_sentence_tokens = []
for index,val in low_ratings.iterrows():
  low_sentence_tokens.append(sent_tokenize(val[0]))

In [ ]:
#Converting to word tokens in one list high ratings
tokenized_high_ratings = []
final_high_tokens = []
for x in high_ratings_list:
  tokenized_high_ratings.append(word_tokenize(x))
for x in tokenized_high_ratings:
  final_high_tokens.append(x)
  

In [ ]:
#Converting to word tokens in one list low ratings
tokenized_low_ratings = []
final_low_tokens = []
for x in low_ratings_list:
  tokenized_low_ratings.append(word_tokenize(x))
for x in tokenized_low_ratings:
  final_low_tokens.append(x)

In [ ]:
#Normalize data
def remove_accents(input_str):
  nfkd_form = unicodedata.normalize('NFKD', input_str)
  only_ascii = nfkd_form.encode('ASCII', "ignore")
  return only_ascii.decode()

  final_low_tokens = remove_accents(paragraph)
  final_high_tokens = remove_accents(paragraph)

In [ ]:
#Instantiate stopwords
my_stopwords = set(stopwords.words("english"))

In [ ]:
#Instantiate punctuation stops
my_punctuation = set(string.punctuation)
print (my_punctuation)

{'<', '>', '|', '*', '!', ',', '{', '"', ';', '#', "'", '_', '\\', ':', '}', ']', '[', '(', '=', '`', '@', '/', '~', '%', ')', '.', '-', '&', '?', '^', '$', '+'}


In [ ]:
extra_word_check = ["``", "''","..."]   

In [ ]:
#Stopwords and Punctuation Low ratings
cleaned_low = []
for element in final_low_tokens:
  for x in element: 
    if x not in my_stopwords and x not in my_punctuation and x not in extra_word_check:
      cleaned_low.append(x)
print (cleaned_low)

['show', 'supplies', 'work', 'start', '8am', 'leave', '10:45', "n't", 'turn', 'till', '4:40.', 'never', 'stay', '4', 'hours', 'simple', 'repair', 'would', 'take', '4', 'days', '3', 'weeks', 'called', 'boss', 'ca', "n't", 'manage', 'people', 'new', 'cabinet', 'vanity', 'faucet', 'installed', 'company', '2', 'months', 'ago', 'totally', 'messed', 'always', 'leaking', 'different', 'spots', 'ask', 'lowes', 'plumber', 're-install', 'everything', 'said', 'call', '4', 'handyman', "n't", 'installed', 'right', 'way', 'sure', 'company', 'sucks', 'unprofessional', 'ugly', 'work', 'hope', 'stop', 'running', 'business', 'soon', 'wanted', 'use', 'groupon', 'closed', 'renovation', 'sure', 'long', 'last', 'may', 'want', 'call', 'first', 'thinking', 'going', 'soon', 'update', 'actually', 'go', 'usually', 'love', 'salon', 'got', 'worst', 'pedicure', 'yesterday', 'shellac', 'already', 'come', 'big', 'toe', 'cuticles', 'hurt', 'scraping', 'polish', 'painted', 'almost', 'forgot', 'top', 'coat', 'shaky', '.s

In [ ]:
#Stopwords and Punctuation High ratings
cleaned_high = []
for element in final_high_tokens:
  for x in element: 
    if x not in my_stopwords and x not in my_punctuation and x not in extra_word_check:
      cleaned_high.append(x)
print (cleaned_high)

['apparently', 'first', 'review', 'company', 'ca', "n't", 'fathom', 'others', 'reveiwed', 'company', '23', 'year', 'old', 'garage', 'door', 'finally', 'gave', 'ghost', 'nite', 'got', 'quotes', 'competitors', 'ranging', '750.00', '1,100.00', 'dollars', 'companies', 'came', 'immediately', 'said', 'needed', 'new', 'door', 'expressed', 'budget', 'limited', 'new', 'door', "n't", 'budget', 'friend', 'said', 'contact', 'aaa', 'action', 'doors', 'contacted', 'josh', 'part', 'owner', 'told', 'budget', 'limited', 'came', 'took', 'one', 'look', 'door', '..said', 'got', 'repaired', 'w/o', 'question..all', 'needed', 'new', 'metal', 'bracing', 'along', 'top', 'middle', 'installed', 'metal', 'bracing', 'less', '45', 'min', '.i', 'operating', 'door', 'again..and', 'saved', 'bunch', 'money', 'happy', 'greg', 'gc', 'glass', 'thought', 'would', 'expensive', 'window', 'replacement', 'actually', 'quick', 'fix', 'minimal', 'cost', 'greg', 'honest', 'knowledgeable', 'certainly', 'call', 'window', 'repair', '

In [ ]:
#Low ratings word count
low_rating_dict = {}
for x in cleaned_low:
  if x not in low_rating_dict:
    low_rating_dict[x] = 0
  if x in low_rating_dict:
    low_rating_dict[x] += 1

In [ ]:
#High ratings word count
high_rating_dict = {}
for x in cleaned_high:
  if x not in high_rating_dict:
    high_rating_dict[x] = 0
  if x in high_rating_dict:
    high_rating_dict[x] += 1

In [ ]:
#Print low ratings word count
sorted_low_rating_count = sorted(low_rating_dict, key=low_rating_dict.get, reverse = False)
for x in sorted_low_rating_count:
  print(x, low_rating_dict[x])

Streaming output truncated to the last 5000 lines.
drainage 1
unclog 1
dimensions 1
compacted 1
sunk 1
trench 1
practically 1
battle 1
quiet 1
landscaper 1
545 1
deposits 1
growing 1
steak 1
unique 1
palm 1
z 1
awarded 1
ava 1
grinder 1
valuable 1
veneer 1
re-order 1
adios 1
378 1
charles 1
inflation 1
proving 1
spds 1
tree 1
affecting 1
8th 1
7:00am 1
hounded 1
37 1
panda 1
wok 1
vague 1
helmets 1
bats 1
funtasticks 1
tucson 1
3br 1
residence 1
elaborate 1
produced 1
adjust 1
replace/aim 1
33 1
optimistically 1
movers 1
anne 1
owen 1
dare 1
repare 1
braised 1
evacuated 1
dclc 1
repour 1
pnc 1
tellers 1
bankers 1
transactions 1
paycheck 1
wise- 1
pinehurst 1
4/23/12 1
39.00 1
4/30/12 1
trevor 1
southwest 1
france 1
fridges 1
.and 1
nuns 1
island 1
vwjettas 1
918..alex 1
volkswagen 1
.im 1
speedy 1
door..incredible 1
relations 1
.its 1
week..no 1
anyone..your 1
midas 1
dated.. 1
hooker 1
excalibur 1
feb. 1
205 1
logged 1
grace 1
rpm 1
human 1
shirts 1
launder 1
ati 1
craving 1
4/7-4/8/1

In [ ]:
#Print high ratings word count
sorted_high_rating_count = sorted(high_rating_dict, key=high_rating_dict.get, reverse = False)
for x in sorted_high_rating_count:
  print(x, high_rating_dict[x])

Streaming output truncated to the last 5000 lines.
eat 1
duplex 1
hills 1
1993 1
tossed 1
jones 1
johnathon 1
non 1
allegedly 1
hv 1
repaired/replaced 1
tubes 1
hoses 1
reports 1
prepares 1
utility 1
plain 1
english 1
shaun 1
handyman/las 1
generate 1
charger 1
overheat 1
scoop 1
vertical 1
reframe 1
gmr 1
regulator 1
tp 1
skillful 1
hastily 1
entryway 1
bothersome 1
reinstall 1
shared 1
re-syncing 1
mortar 1
armstrong 1
jazz 1
gardener 1
shines 1
evaluate 1
returns 1
'hustlers 1
gifted 1
artash 1
surprisingly 1
handiest 1
penguin 1
3200 1
onsite 1
110 1
up-charging 1
badass 1
vacuums 1
marc 1
cakebread 1
pre 1
putting 1
parrott 1
smartest 1
receptive 1
durango 1
brandler 1
jacob 1
breakdown 1
darla 1
prioritizing 1
obnoxious 1
slams 1
damn 1
race 1
suit 1
modified 1
lamont 1
communicate 1
underground 1
pvc 1
chlorinator 1
limiter 1
co 1
jgm 1
typical 1
transmition 1
heart 1
lonnie 1
demonstrate 1
consumers 1
powerhead 1
aurthur 1
vaccuum 1
vacworks 1
newmarket 1
berg 1
became 1
pertai

In [ ]:
#function to prep for bigram or trigram
def anagram_prep(my_list, desired_len):
  if len(my_list) % desired_len == 0:
    return my_list
  if len(my_list) % desired_len == 1:
    my_list.remove(my_list[len(my_list)-1])
    return my_list
  if len(my_list) % desired_len == 2:
    my_list.remove(my_list[len(my_list)-1])
    my_list.remove(my_list[len(my_list)-1])
    return my_list
  if len(my_list) % desired_len > 2:
    return my_list


In [ ]:
#function to create bigrams or trigrams
def anagram_creation(my_list, desired_len):
  def anagram_prep(my_list, desired_len):
    if len(my_list) % desired_len == 0:
      return my_list
    if len(my_list) % desired_len == 1:
      my_list.remove(my_list[len(my_list)-1])
      return my_list
    if len(my_list) % desired_len == 2:
      my_list.remove(my_list[len(my_list)-1])
      my_list.remove(my_list[len(my_list)-1])
      return my_list
    if len(my_list) % desired_len > 2:
      return my_list

  
  working_list = anagram_prep(my_list, desired_len)
  if desired_len == 2:
    output_list = []
    times_calculations_run = len(working_list) / 2
    my_counter = 0
    bigram_one = 0
    bigram_two = 1

    while my_counter != times_calculations_run:
      bigram_element = working_list[bigram_one] + " " + working_list[bigram_two]
      output_list.append(bigram_element)
      my_counter += 1
      bigram_one += 2
      bigram_two += 2

  if desired_len == 3:
    output_list = []
    times_calculations_run = len(working_list) / 3
    my_counter = 0
    trigram_one = 0
    trigram_two = 1
    trigram_three = 2

    while my_counter != times_calculations_run:
      trigram_element = working_list[trigram_one] + " " + working_list[trigram_two] + " " + working_list[trigram_three]
      output_list.append(trigram_element)
      my_counter += 1
      trigram_one += 3
      trigram_two += 3
      trigram_three += 3

  return output_list


In [ ]:
#Delete ME
anagram_test = ["turtle", "fish", "cat", "frog","lion", "gorilla", "zebra", "bird"]
result = anagram_creation(anagram_test, 2)

In [ ]:
#DELETE ME
#my_result

In [ ]:
#Trigram document High Ratings
trigram_document_high = []
for x in final_high_tokens:
  trigram_document_high.append(anagram_creation(x,3))

In [ ]:
#Bigram document High Ratings
bigram_document_high = []
for x in final_high_tokens:
  bigram_document_high.append(anagram_creation(x,2))

In [ ]:
#Trigram document Low Ratings
trigram_document_low = []
for x in final_low_tokens:
  trigram_document_low.append(anagram_creation(x,3))

In [ ]:
#Bigram document Low Ratings
bigram_document_low = []
for x in final_low_tokens:
  bigram_document_low.append(anagram_creation(x,2))

In [ ]:
#Bigram Low Ratings Dictionary Word Count
bigram_low_dict = {}
for ele in bigram_document_low:
  for word in ele:
    if word not in bigram_low_dict:
      bigram_low_dict[word] = 0
    if word in bigram_low_dict:
      bigram_low_dict[word] += 1

In [ ]:
#Print dictionary
for k,v in bigram_low_dict.items():
  if v > 6:
    print (k,v)

show up 11
with no 12
do the 15
! they 18
and do 13
more than 21
a simple 11
repair that 9
3 weeks 11
i called 69
at all 30
i had 112
from this 24
months ago 12
, it 30
up , 10
again , 12
they said 34
did n't 74
company is 7
they do 36
wanted to 19
and they 83
not sure 11
to call 19
you are 19
. will 7
i can 21
but i 24
, my 10
, so 24
. they 149
of a 30
there is 13
. not 17
i took 16
for repair 20
when i 66
the staff 9
refused to 21
after i 13
they would 41
repair it 16
for a 100
it was 106
in the 103
i just 20
think they 9
i hired 7
them to 35
leak . 11
their work 14
on the 88
the leak 13
where they 9
cost me 9
over $ 31
for my 12
the wrong 8
to repair 66
work . 32
said they 19
all . 9
they were 65
they are 74
. i 355
did not 54
. this 43
they also 8
this was 27
the only 10
just a 10
from a 7
company that 16
me . 24
out there 7
. do 32
with this 24
! ! 245
! do 10
not use 15
came out 38
to my 29
repair my 15
ended up 11
to fix 55
said that 18
they did 47
n't do 18
it and 21
that i 43

In [ ]:
#Trigram low Ratings Dictionary Word Count
trigram_low_dict = {}
for ele in trigram_document_low:
  for word in ele:
    if word not in trigram_low_dict:
      trigram_low_dict[word] = 0
    if word in trigram_low_dict:
      trigram_low_dict[word] += 1

In [ ]:
#Bigram high Ratings Dictionary Word Count
bigram_high_dict = {}
for ele in bigram_document_high:
  for word in ele:
    if word not in bigram_high_dict:
      bigram_high_dict[word] = 0
    if word in bigram_high_dict:
      bigram_high_dict[word] += 1

In [ ]:
#Trigram high Ratings Dictionary Word Count
trigram_high_dict = {}
for ele in trigram_document_high:
  for word in ele:
    if word not in trigram_high_dict:
      trigram_high_dict[word] = 0
    if word in trigram_high_dict:
      trigram_high_dict[word] += 1

In [ ]:
#speech_values = pos_tag(final_cleaned_Kanye)
speech_low_ratings = []
speech_high_ratings = []
for x in final_low_tokens:
  speech_low_ratings.append(pos_tag(x))
for x in final_high_tokens:
  speech_high_ratings.append(pos_tag(x))
  


In [ ]:
#Low Rating dictionary word type break down
low_adj_dict = {}
low_verb_dict = {}
low_noun_dict = {}
for ele in speech_low_ratings:
  for x in ele:
    if x[1] == "JJ" or x[1] == "JJR" or x[1] == "JJS":
      if x[0] not in low_adj_dict:
        low_adj_dict[x[0]] = 1
      if x[0] in low_adj_dict:
        low_adj_dict[x[0]] += 1
    if x[1] == "NN" or x[1] == "NNP" or x[1] == "NNS":
      if x[0] not in low_noun_dict:
        low_noun_dict[x[0]] = 1
      if x[0] in low_noun_dict:
        low_noun_dict[x[0]] += 1
    if x[1] == "VB" or x[1] == "VBD" or x[1] == "VBG" or x[1] == "VBN" or x[1] == "VBP" or x[1]== "VBZ":
      if x[0] not in low_verb_dict:
        low_verb_dict[x[0]] = 1
      if x[0] in low_verb_dict:
        low_verb_dict[x[0]] += 1



In [ ]:
#HIgh Rating dictionary word type break down
high_adj_dict = {}
high_verb_dict = {}
high_noun_dict = {}
for ele in speech_high_ratings:
  for x in ele:
    if x[1] == "JJ" or x[1] == "JJR" or x[1] == "JJS":
      if x[0] not in high_adj_dict:
        high_adj_dict[x[0]] = 1
      if x[0] in high_adj_dict:
        high_adj_dict[x[0]] += 1
    if x[1] == "NN" or x[1] == "NNP" or x[1] == "NNS":
      if x[0] not in high_noun_dict:
        high_noun_dict[x[0]] = 1
      if x[0] in high_noun_dict:
        high_noun_dict[x[0]] += 1
    if x[1] == "VB" or x[1] == "VBD" or x[1] == "VBG" or x[1] == "VBN" or x[1] == "VBP" or x[1]== "VBZ":
      if x[0] not in high_verb_dict:
        high_verb_dict[x[0]] = 1
      if x[0] in high_verb_dict:
        high_verb_dict[x[0]] += 1

In [ ]:
print (high_adj_dict)

{'first': 69, 'old': 69, 'other': 128, 'nite': 2, 'new': 228, 'contact': 3, 'aaa': 2, 'top': 20, 'middle': 2, 'less': 41, 'happy': 90, 'expensive': 20, 'quick': 82, 'minimal': 4, 'honest': 108, 'knowledgeable': 81, 'further': 12, 'odd': 3, 'personable': 15, 'professional': 278, 'exemplary': 3, 'necessary': 19, 'many': 44, 'great': 360, 'most': 31, 'scottsdale': 3, 'disjointed': 2, 'cozy': 4, 'giants/padre': 2, 'sleepy': 2, 'genuine': 5, 'appreciated': 2, 'little': 29, 'fair': 68, 'critical': 4, 'several': 52, 'recommended': 16, 'few': 83, 'busy': 20, 'reasonable': 84, 'i': 294, 'straightforward': 4, 'worst': 6, 'az': 6, 'excellent': 81, 'needed': 5, 'good': 144, 'polite': 30, 'thorough': 30, 'recommend': 43, 'second': 24, 'large': 20, 'sectional': 3, 'outstanding': 23, 'happier': 21, 'next': 79, 'satisfied': 19, 'technician': 24, 'following': 11, 'experienced': 5, 'wonderful': 38, 'minor': 16, 'different': 29, 'main': 12, 'efficient': 41, 'nice': 38, 'sorry': 6, 'moon': 2, 'larger': 3,

In [ ]:
speech_low_ratings[2]

[('wanted', 'VBN'),
 ('to', 'TO'),
 ('use', 'VB'),
 ('my', 'PRP$'),
 ('groupon', 'NN'),
 (',', ','),
 ('and', 'CC'),
 ('they', 'PRP'),
 ('were', 'VBD'),
 ('closed', 'VBN'),
 ('for', 'IN'),
 ('renovation', 'NN'),
 ('not', 'RB'),
 ('sure', 'JJ'),
 ('how', 'WRB'),
 ('long', 'JJ'),
 ('that', 'DT'),
 ('will', 'MD'),
 ('last', 'JJ'),
 (',', ','),
 ('but', 'CC'),
 ('you', 'PRP'),
 ('may', 'MD'),
 ('want', 'VB'),
 ('to', 'TO'),
 ('call', 'VB'),
 ('first', 'RB'),
 ('if', 'IN'),
 ('you', 'PRP'),
 ('are', 'VBP'),
 ('thinking', 'VBG'),
 ('of', 'IN'),
 ('going', 'VBG'),
 ('soon', 'RB'),
 ('.', '.'),
 ('will', 'MD'),
 ('update', 'VB'),
 ('when', 'WRB'),
 ('i', 'NN'),
 ('can', 'MD'),
 ('actually', 'RB'),
 ('go', 'VB')]

In [ ]:
#TFID
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 3), stop_words='english')

In [ ]:
#TFID 
features = tfidf.fit_transform(total_ratings.Reviews).toarray()
labels = total_ratings.Ratings
features.shape

(2417, 2980)

In [ ]:

# EXAMPLE WEBSITE: https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f
N = 2
for Ranking, Ratings in sorted(total_ratings.Ratings.items()):
  features_chi2 = chi2(features, labels == Ratings)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  trigrams = [v for v in feature_names if len(v.split(' ')) == 3]
  print("# '{}':".format(Ranking))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))
  print("  . Most correlated trigrams:\n. {}".format('\n. '.join(trigrams[-N:])))


Streaming output truncated to the last 5000 lines.
# '3228':
  . Most correlated unigrams:
. highly
. professional
  . Most correlated bigrams:
. stay away
. highly recommend
  . Most correlated trigrams:
. highly recommend company
. did great job
# '3229':
  . Most correlated unigrams:
. highly
. professional
  . Most correlated bigrams:
. stay away
. highly recommend
  . Most correlated trigrams:
. highly recommend company
. did great job
# '3230':
  . Most correlated unigrams:
. highly
. professional
  . Most correlated bigrams:
. stay away
. highly recommend
  . Most correlated trigrams:
. highly recommend company
. did great job
# '3231':
  . Most correlated unigrams:
. highly
. professional
  . Most correlated bigrams:
. stay away
. highly recommend
  . Most correlated trigrams:
. highly recommend company
. did great job
# '3232':
  . Most correlated unigrams:
. highly
. professional
  . Most correlated bigrams:
. stay away
. highly recommend
  . Most correlated trigrams:
. highl

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(total_ratings['Reviews'], total_ratings['Ranking'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [ ]:
#ExtraTreesClassifier
extra_clf = ExtraTreesClassifier(bootstrap = True, max_leaf_nodes = 100, n_estimators = 350)
extra_clf.fit(X_train_tfidf, y_train)

ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=100, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=350,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [ ]:
extra_pred = extra_clf.predict(count_vect.transform(X_test))

In [ ]:
recall_score(y_test, extra_pred, average='weighted')

0.8876033057851239

In [ ]:
#Extra trees Confusion Matrix 
extra_cm = confusion_matrix(y_test, extra_pred)
print (extra_cm)

[[259  48]
 [ 20 278]]


In [ ]:
precision_score(y_test, extra_pred, average='weighted')

0.8911000511202012

In [ ]:
accuracy_score(y_test, extra_pred)

0.8876033057851239

In [ ]:
f1_score(y_test, extra_pred, average="weighted")

0.8874397812343825

In [ ]:
#Random Forest Accuracy Score
accuracy_score(y_test, extra_pred)

0.8876033057851239

In [ ]:
#Random Forest
r_clf = RandomForestClassifier(max_depth=6, random_state=0)
r_clf.fit(X_train_tfidf, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=6, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
#Randoms prediction
print(r_clf.predict(count_vect.transform(["Mr.jones did a negative job on the cement"])))

['positive']


In [ ]:
#Random prediction
r_pred = r_clf.predict(count_vect.transform(X_test))

In [ ]:
recall_score(y_test, r_pred, average='weighted')

0.8495867768595041

In [ ]:
#Random Forest Confusion Matrix 
r_cm = confusion_matrix(y_test, r_pred)
print (r_cm)

[[246  61]
 [ 30 268]]


In [ ]:
X_train_tfidf[3]

<1x7711 sparse matrix of type '<class 'numpy.float64'>'
	with 31 stored elements in Compressed Sparse Row format>

In [ ]:
f1_score(y_test, r_pred, average="weighted")

0.8493061467299651

In [ ]:
#Random Forest Accuracy Score
accuracy_score(y_test, r_pred)

0.8495867768595041

In [ ]:
precision_score(y_test, y_pred, average='weighted')

0.9115652075217453

In [ ]:
precision_score(y_test, r_pred, average='weighted')

0.8535176118138021

In [ ]:
#BAYES prediction
print(clf.predict(count_vect.transform(["Greg the handyman was very professional."])))

['positive']


In [ ]:
#Gradient Boosting
g_clf = GradientBoostingClassifier(random_state=0, learning_rate=1, max_depth=15, min_samples_leaf = 100)
g_clf.fit(X_train_tfidf, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=1, loss='deviance', max_depth=15,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=100, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
#GRADEIENT PREDICT
g_predict = g_clf.predict(count_vect.transform(X_test))

In [ ]:
#GRADIENT MATRIX
g_cm = confusion_matrix(y_test, g_predict)
print (g_cm)

[[196 111]
 [ 38 260]]


In [ ]:
#GRADIENT F1 SCORE
f1_score(y_test, g_predict, average='weighted')

0.750539863201789

In [ ]:
#ACCURACY_SCORE Gradient
accuracy_score(y_test, g_predict)

0.7537190082644628

In [ ]:
recall_score(y_test, g_predict, average='weighted')

0.7537190082644628

In [ ]:
#GRADIENT PREDICTION
print(g_clf.predict(count_vect.transform(["Mr.jones did a negative job on the cement"])))

['positive']


In [ ]:
#BAYES Prediction
y_pred = clf.predict(count_vect.transform(X_test))

In [ ]:
#CONFUSION MATRIX BAYES
cm = confusion_matrix(y_test, y_pred)
print (cm)

[[287  20]
 [ 34 264]]


In [ ]:
#ACCURACY_SCORE BAYES
accuracy_score(y_test, y_pred)

0.9107438016528926

In [ ]:
recall_score(y_test, y_pred, average='weighted')

0.9107438016528926

In [ ]:
f1_score(y_test, y_pred, average="weighted")

0.9106651674246993

Over sample negatives / Undersample positives / Improve recall
Gradient Boost Classifier SKLearn 




In [ ]:
print (x_pred)

NameError: ignored

In [ ]:
"""
#IGNORE/ BUT DON"T DELETE
for x in real_final_Kanye:
  if x == "'":
    real_final_Kanye.remove(x)
  if x == "i":
    real_final_Kanye.remove(x)
  if x == "’":
    real_final_Kanye.remove(x)
"""

In [ ]:
#IGNORE/DON"T DELETE
#Gets word types of all the words Parts of Speech
#speech_values = pos_tag(final_cleaned_Kanye)

Notes from HAMID
scikit learn TFID